In [1]:
import numpy as np
import pandas as pd
import math
import scipy
import sys
from tqdm import tqdm
import functools
import networkx as nx
import itertools

import portion as P
import re
import datetime
import queue

# interval = P.empty()
# for i, j in [(0, 1), (2, 4), (6, 7), (3, 5)]:
#     interval = interval | P.closed(i, j)
# print(interval)

In [2]:
class Object3d():
    def __init__(self) -> None:
        self.cubes = []
        self.G=nx.Graph()

    def __repr__(self) -> str:
        if self.cubes:
            return '\n'.join([str(c) for c in self.cubes])
        return 'empty Object3d'

    def add_cube(self,node):
        self.G.add_node(node)
        self.cubes.append(node)

        for c in self.cubes:
            if self.find_distance(node,c) == 1:
                self.G.add_edge(node,c)

    def get_distance(self, node):
        #gets min Manhattan distance between self.cubes and the given input
        x,y,z = node
        return min([abs(x-i)+abs(y-j)+abs(z-k) for i,j,k in self.cubes])

    def find_distance(self, n1, n2):
        #gets min Manhattan distance between two cubes
        x,y,z = n1
        i,j,k = n2
        return abs(x-i)+abs(y-j)+abs(z-k)
    
    def get_surface(self):
        return sum([6-self.G.degree(n) for n in self.cubes])

    def get_exterior_surface(self):
        space_range = []
        x = []
        y = []
        z = []
        for c in self.cubes:
            x.append(c[0])
            y.append(c[1])
            z.append(c[2])
        for l in [x,y,z]:
            space_range.append((l.min(),l.max()))
        


In [3]:
file1 = open('18.txt', 'r')
lines = file1.readlines()

cubes = []

for l in lines:
    x,y,z = l.replace('\n','').split(',')
    cubes.append((int(x),int(y),int(z)))
    # print(x,y,z)

print(f'loaded {len(cubes)} cubes')

objects = []

while cubes:
    obj = Object3d()
    obj.add_cube(cubes.pop(0))

    object_expanded = True
    while object_expanded:
        expand_by_idx = []
        object_expanded = False

        for i,cube in enumerate(cubes):
            if obj.get_distance(cube) == 1:
                expand_by_idx.append(i)
        if expand_by_idx:
            # print(f'expanding by {expand_by_idx}')
            for i in sorted(expand_by_idx, reverse=True):
                obj.add_cube(cubes.pop(i))
            object_expanded = True
        # print(f'object has {len(obj.cubes)} cubes:\n{obj}\n')
    # print(f'object has {len(obj.cubes)} cubes:\n{obj}\n')
    print(f'object has {len(obj.cubes)} cubes')
    objects.append(obj)


loaded 13 cubes
object has 8 cubes
object has 1 cubes
object has 1 cubes
object has 1 cubes
object has 1 cubes
object has 1 cubes


In [4]:
##code for one object

# obj = objects[0]
# obj.get_surface()

In [5]:
sides = 0
for obj in objects:
    sides += obj.get_surface()

sides

64

# Second part

Find all interior cubes, add them to the input and run solution for part 1

In [6]:
file1 = open('18.txt', 'r')
lines = file1.readlines()

cubes = []

for l in lines:
    x,y,z = l.replace('\n','').split(',')
    cubes.append((int(x),int(y),int(z)))

space_range = []
x = []
y = []
z = []
for c in cubes:
    x.append(c[0])
    y.append(c[1])
    z.append(c[2])

for l in [x,y,z]:
    space_range.append((min(l),max(l)))

    
def get_non_object_neighbors(c):
    ret = []
    x,y,z = c
    for i,j,k in [[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]]:
        new_cube = (x+i,y+j,z+k)
        if new_cube not in cubes:
            ret.append(new_cube)
    return ret

# for c in self.cubes:
## add interior cubes using dijkstra's

test_cubes = [(x,y,z) for x in range(space_range[0][0],space_range[0][1]+1) for y in range(space_range[1][0],space_range[1][1]+1) for z in range(space_range[2][0],space_range[2][1]+1)]

exterior_cubes = set()

interior_cubes = set()
for c in tqdm(test_cubes):
    if c in cubes or c in interior_cubes or c in exterior_cubes:
        continue
    # print(c)
    expanded_set = set()
    expanded_set_range = [[c[0],c[0]],[c[1],c[1]],[c[2],c[2]]]
    current_set = {c}
    while True:
        tmp_set = set()
        for cube in current_set:
            tmp_set = tmp_set | set(get_non_object_neighbors(cube))
        expanded_set = expanded_set | current_set
        for cube in current_set:
            #update expanded_set_range
            for i in range(3):
                if cube[i] < expanded_set_range[i][0]:
                    expanded_set_range[i][0] = cube[i]
                if cube[i] > expanded_set_range[i][1]:
                    expanded_set_range[i][1] = cube[i]

        current_set = tmp_set
        # print(f'{expanded_set=}')
        
        if not current_set or len(current_set) == len(current_set & expanded_set):
            #this is interior pocket
            # print(f'Found and interior pocket with the size of {len(expanded_set)}')
            interior_cubes = interior_cubes | expanded_set
            break
        if any([expanded_set_range[i][0] < space_range[i][0] for i in range(3)] + [expanded_set_range[i][1] > space_range[i][1] for i in range(3)]):
            #we're on the outside
            exterior_cubes = exterior_cubes | expanded_set
            break

        if len(expanded_set) > (len(cubes)//6+2)**(3/2):
            print(f'{len(expanded_set)=}, {len(cubes)=}')

        if len(exterior_cubes & expanded_set) > 0:
            #we're on the outside
            exterior_cubes = exterior_cubes | expanded_set
            break

        # if len(expanded_set) > 300:
        #     print(len(expanded_set))


# test_cubes

100%|██████████| 54/54 [00:00<00:00, 51828.93it/s]


In [7]:
len(set(interior_cubes))

1

In [9]:
print(f'loaded {len(cubes + list(interior_cubes))} cubes')

objects = []

cubes2 = cubes + list(interior_cubes)

while cubes2:
    obj = Object3d()
    obj.add_cube(cubes2.pop(0))

    object_expanded = True
    while object_expanded:
        expand_by_idx = []
        object_expanded = False

        for i,cube in enumerate(cubes2):
            if obj.get_distance(cube) == 1:
                expand_by_idx.append(i)
        if expand_by_idx:
            # print(f'expanding by {expand_by_idx}')
            for i in sorted(expand_by_idx, reverse=True):
                obj.add_cube(cubes2.pop(i))
            object_expanded = True
    print(f'object has {len(obj.cubes)} cubes')
    objects.append(obj)

loaded 14 cubes
object has 14 cubes


In [10]:
sides = 0
for obj in objects:
    sides += obj.get_surface()

sides

58